In [2]:
from syft.core.adp.data_subject import DataSubject
import syft as sy
import jax
import numpy as np
import functools

from typing import List
from typing import Tuple


In [1]:
class MultiRowPhiTensor(): 
    """
    This class acts like a wrapper over multiple PhiTensors that will offer an helpful API
    instead of having the user to work with multiple PhiTensors of the same structure 
    """
    phi_tensors: np.array[sy.PhiTensor]
    
    def __init__(self, phi_tensors: List[sy.PhiTensor]):
        phi_tensors = np.array(phi_tensors)
        
    def execute_op(self, op, args, kwargs) -> MultiRowPhiTensor:
        return MultiRowPhiTensor(self.phi_tensors.apply(op, args, kwargs))
        
    def reduce_op(self, op, args, kwargs) -> sy.GammaTensor:
        return functools.reduce(op, self.phi_tensors)
    
    def to_flex(self):
        return MultiFlexPhiTensor(self.phi_tensors)
        
        
class MultiFlexPhiTensor():
    """
    This structure is closer to the old PhiTensor, 
    as each value has mapped a DataSubject or DataSubjectArray
    """
    data_subjects_indices: np.array
    data: np.array
    lower_bounds: np.array
    upper_bounds: np.array
    shallow_compute: bool = False
    
    
    def __init__(self, phi_tensors: List[sy.PhiTensor]):
        data_subjects = [phi_tensor.data_subject for phi_tensor in phi_tensors]
        data = np.array([phi_tensor.data for phi_tensor in phi_tensors])
        shape = (len(phi_tensors) *phi_tensors[0].shape)
        lower_bounds = sy.lazyrepeatarray([phi_tensor.lower_bound for phi_tensor in self.phi_tensors], shape=shape)
        upper_bounds = sy.lazyrepeatarray([phi_tensor.upper_bound for phi_tensor in self.phi_tensors], shape=shape)
    
    def __init__(self, data, data_subjects_indices, lower_bounds_array, upper_bounds_array):
        
        if not data.shape == data_subjects_indices.shape == lower_bounds_array.shape == upper_bounds_array.shape:
            raise Exception
        
        self.data = data
        self.data_subjects_indices == data_subjects_indices
        self.lower_bounds = lower_bounds_array
        self.upper_bounds = upper_bounds_array
    
    def ispublic(other):
        return False
    
    def __add__(self, other):
        if ispublic(other):
            data = self.data + other
            lower_bounds = self.lower_bounds + other
            upper_bounds = self.upper_bounds + other
            data_subjects_indices = self.data_subjects_indices
        else: 
            data_subjects_match = (self.data_subjects_indices == other.data_subject_indices).all()
            if data_subjects_match:
                data_subjects_indices = self.data_subjects_indices
                data = self.data + other.data
                lower_bounds = self.lower_bounds + other.lower_bounds
                upper_bounds = self.upper_bounds + other.upper_bounds
            else:
                return self.gamma + other.gamma
        
        return MultiFlexPhiTensor(...)    
    
    
    

NameError: name 'np' is not defined

In [ ]:
# for simple operations:
phi_tensors_obj = MultiRowPhiTensor(...)
phi_tensors_obj *= 2
phi_tensors_obj.sum()

In [ ]:
phi_tensors_obj = MultiFlexPhiTensor(...)
phi_tensors_obj.T 
phi_tensors_obj @ phi_tensors_obj

In [ ]:
class PhiTensor():
    child: comp_array
    bounds: comp_array
    data_subject: sy.DataSubject
    
    def __array_interface__():
        

In [ ]:
x,y,z = PhiTensor(...)
np.array(x,y,z)